# Import Data 
from Dask-Batches and save it as Dataframe

In [2]:
import pandas as pd
import os

In [3]:
data_path = "../na_mentions_final/"
data_folders = sorted([folder for folder in os.listdir(data_path)])
output_path = "data/"

In [4]:
data_folders

['00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']

## Test for Folder 0

In [5]:
list00 = []
for folder in data_folders[:1]:
    files = [file for file in os.listdir(os.path.join(data_path, folder))]
    print(files)

    for file in files:
        df = pd.read_json(os.path.join(data_path, folder, file))
        print(len(df))
        list00.append(df)

['1_4.jsonl.bz2', '7_5.jsonl.bz2', '5_2.jsonl.bz2', '3_4.jsonl.bz2', '1_7.jsonl.bz2', '8_5.jsonl.bz2', '1_8.jsonl.bz2', '4_4.jsonl.bz2', '6_1.jsonl.bz2', '4_6.jsonl.bz2', '0_0.jsonl.bz2', '3_2.jsonl.bz2', '1_5.jsonl.bz2', '4_5.jsonl.bz2', '6_8.jsonl.bz2', '5_8.jsonl.bz2', '0_7.jsonl.bz2', '9_2.jsonl.bz2', '5_4.jsonl.bz2', '8_2.jsonl.bz2', '3_6.jsonl.bz2', '0_4.jsonl.bz2', '2_2.jsonl.bz2', '2_0.jsonl.bz2', '5_3.jsonl.bz2', '4_2.jsonl.bz2', '0_9.jsonl.bz2', '3_5.jsonl.bz2', '4_8.jsonl.bz2', '6_3.jsonl.bz2', '2_5.jsonl.bz2', '1_3.jsonl.bz2', '8_3.jsonl.bz2', '4_3.jsonl.bz2', '8_7.jsonl.bz2', '5_0.jsonl.bz2', '9_4.jsonl.bz2', '8_8.jsonl.bz2', '2_7.jsonl.bz2', '3_3.jsonl.bz2', '0_3.jsonl.bz2', '1_9.jsonl.bz2', '3_0.jsonl.bz2', '2_8.jsonl.bz2', '0_8.jsonl.bz2', '2_4.jsonl.bz2', '1_2.jsonl.bz2', '9_9.jsonl.bz2', '2_9.jsonl.bz2', '5_6.jsonl.bz2', '4_9.jsonl.bz2', '9_8.jsonl.bz2', '0_2.jsonl.bz2', '1_0.jsonl.bz2', '2_6.jsonl.bz2', '9_7.jsonl.bz2', '9_1.jsonl.bz2', '8_4.jsonl.bz2', '7_6.jsonl.bz

## Importing Everything

In [6]:
list_all = []

for folder in data_folders:
    files = [file for file in os.listdir(os.path.join(data_path, folder))]
    
    print(f"Number of files contained in folder {folder}:", len(files))

    for file in files:
        df = pd.read_json(os.path.join(data_path, folder, file))
        list_all.append(df)

mentions_all = pd.concat(list_all, axis=0, ignore_index=True)

#dropping articleauthor
len_before = len(mentions_all)
mentions_all = mentions_all[mentions_all['entity'] != "pers.ind.articleauthor"]
print(f"Dropped {len_before - len(mentions_all)} rows which contain pers.ind.articleauthor mention")

#infer new columns
mentions_all['split_id'] = mentions_all['id'].str.split("-")
mentions_all['article'] = mentions_all['id'].apply(lambda x: x.split(":")[0])
mentions_all['newspaper'] = mentions_all['split_id'].apply(lambda x: x[0])
mentions_all['date'] = mentions_all['split_id'].apply(lambda x: "-".join(x[1:4]))
mentions_all['year'] = mentions_all['split_id'].apply(lambda x: x[1]).astype(int)
mentions_all['decade'] = mentions_all['year'] // 10 * 10
mentions_all['agency'] = mentions_all['entity'].str.replace("org.ent.pressagency.", "", regex=False)
mentions_all['language'] = mentions_all['id'].apply(lambda x: x[-2:])
#newspapers from Switzerland are uppercase, from Luxembourg lowercase
mentions_all['country'] = ["CH" if entry.isupper() else "LU" for entry in mentions_all['newspaper']]
#correct typo
#mentions_all['agency'] = mentions_all['agency'].str.replace("ATB", "ATS", regex=False)
mentions_all = mentions_all.drop(columns='split_id')

Number of files contained in folder 00: 100


Number of files contained in folder 01: 100
Number of files contained in folder 02: 100
Number of files contained in folder 03: 100
Number of files contained in folder 04: 100
Number of files contained in folder 05: 100
Number of files contained in folder 06: 100
Number of files contained in folder 07: 100
Number of files contained in folder 08: 100
Number of files contained in folder 09: 100
Number of files contained in folder 10: 100
Number of files contained in folder 11: 100
Number of files contained in folder 12: 100
Number of files contained in folder 13: 100
Number of files contained in folder 14: 100
Number of files contained in folder 15: 100
Number of files contained in folder 16: 100
Number of files contained in folder 17: 100
Number of files contained in folder 18: 100
Number of files contained in folder 19: 100
Number of files contained in folder 20: 100
Dropped 237520 rows which contain pers.ind.articleauthor mention


In [7]:
mentions_all.head()

,entity,surface,qid,lSentenceOffset,rSentenceOffset,sentence_idx:,lArticleOffset,rArticleOffset,id,article,newspaper,date,year,decade,agency,language,country
0,org.ent.pressagency.Havas,Havas,Q2826560,30,35,1,84,89,LLE-1939-11-27-a-i0076:1:30:35:84:89:newsag:be...,LLE-1939-11-27-a-i0076,LLE,1939-11-27,1939,1930,Havas,fr,CH
1,org.ent.pressagency.UP-UPI,United Press,Q493845,2,14,7,595,607,LLE-1939-11-27-a-i0076:7:2:14:595:607:newsag:b...,LLE-1939-11-27-a-i0076,LLE,1939-11-27,1939,1930,UP-UPI,fr,CH
2,org.ent.pressagency.UP-UPI,United Press,Q493845,0,12,10,1364,1376,LLE-1939-11-27-a-i0076:10:0:12:1364:1376:newsa...,LLE-1939-11-27-a-i0076,LLE,1939-11-27,1939,1930,UP-UPI,fr,CH
3,org.ent.pressagency.UP-UPI,United Press,Q493845,2,14,16,2181,2193,LLE-1939-11-27-a-i0076:16:2:14:2181:2193:newsa...,LLE-1939-11-27-a-i0076,LLE,1939-11-27,1939,1930,UP-UPI,fr,CH
4,org.ent.pressagency.UP-UPI,Press,Q493845,48,53,18,2531,2536,LLE-1939-11-27-a-i0076:18:48:53:2531:2536:news...,LLE-1939-11-27-a-i0076,LLE,1939-11-27,1939,1930,UP-UPI,fr,CH


In [8]:
print(f"Languages in corpus: {mentions_all['language'].unique()}")
mentions_fr = mentions_all[mentions_all['language']=="fr"]
mentions_de = mentions_all[mentions_all['language']=="de"]

Languages in corpus: ['fr' 'de']


In [9]:
articles = mentions_all.groupby(['article']).agg({
    'agency': list,
    'surface': list,
    'id': list,
    'sentence_idx:': list,
    'language': 'first',
    'newspaper': 'first',
    'country': 'first',
    'year': 'first',
    'decade': 'first',
}).reset_index()

articles_fr = articles[articles['language']=="fr"]
articles_de = articles[articles['language']=="de"]

In [10]:
articles.head()

,article,agency,surface,id,sentence_idx:,language,newspaper,country,year,decade
0,BLB-1846-03-28-a-i0002,[Reuters],[Reuel],[BLB-1846-03-28-a-i0002:88:0:5:13580:13585:new...,[88],de,BLB,CH,1846,1840
1,BLB-1846-10-10-a-i0003,[Havas],[Saas],[BLB-1846-10-10-a-i0003:49:0:4:6901:6905:newsa...,[49],de,BLB,CH,1846,1840
2,BLB-1847-02-13-a-i0001,[unk],[pp],[BLB-1847-02-13-a-i0001:168:0:2:13072:13074:ne...,[168],de,BLB,CH,1847,1840
3,BLB-1847-05-22-a-i0003,[SPK-SMP],[sp],[BLB-1847-05-22-a-i0003:37:0:2:2653:2655:newsa...,[37],de,BLB,CH,1847,1840
4,BLB-1847-07-03-a-i0001,[SPK-SMP],[resp],[BLB-1847-07-03-a-i0001:11:40:44:3285:3289:new...,[11],de,BLB,CH,1847,1840


## Save Data

In [11]:
pd.to_pickle(articles_fr, output_path + "articles_fr.pkl")
pd.to_pickle(articles_de, output_path + "articles_de.pkl")
pd.to_pickle(mentions_fr, output_path + "mentions_fr.pkl")
pd.to_pickle(mentions_de, output_path + "mentions_de.pkl")